In [1]:
!pip install arxiv

  Using cached arxiv-2.1.3-py3-none-any.whl.metadata (6.1 kB)
  Using cached feedparser-6.0.11-py3-none-any.whl.metadata (2.4 kB)
  Using cached sgmllib3k-1.0.0-py3-none-any.whl
Using cached arxiv-2.1.3-py3-none-any.whl (11 kB)
Using cached feedparser-6.0.11-py3-none-any.whl (81 kB)


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [4]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [5]:
wiki.name

'wikipedia'

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()

documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever
retriever

In [8]:
# Required installations
# !pip install langchain langchain_community transformers faiss-cpu

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load documents from the web
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

# Use Hugging Face embeddings (e.g., using all-MiniLM-L6-v2 model)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create a FAISS vector store
vectordb = FAISS.from_documents(documents, embeddings)

# Set up retriever
retriever = vectordb.as_retriever()

# Use retriever for downstream tasks
print(retriever)


tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024E2F3E2E40> search_kwargs={}


In [9]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")


In [10]:
retriever_tool.name

'langsmith_search'

In [11]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [12]:
tools=[wiki,arxiv,retriever_tool]

In [26]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Choose a freely available public model
model_name = "EleutherAI/gpt-neo-2.7B"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", load_in_8bit=True)

# Move model to GPU if available, else CPU
model.to("cuda" if torch.cuda.is_available() else "cpu")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install 'accelerate>=0.26.0'`

In [27]:
# Create a text generation pipeline
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0
)


Device set to use cpu


In [28]:
# Integrate Hugging Face pipeline with LangChain
llm = HuggingFacePipeline(pipeline=hf_pipeline)


C:\Users\tanvi\AppData\Local\Temp\ipykernel_6604\2753922783.py:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [29]:
# Get the prompt to use
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages



# Create the agent with Hugging Face model
from langchain.agents import initialize_agent, AgentType

# Initialize the agent with Hugging Face LLM and tools
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Create Agent Executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

f:\langchain-chatbot\venv\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
C:\Users\tanvi\AppData\Local\Temp\ipykernel_6604\4147276551.py:11: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [30]:
# Test the agent executor
response1 = agent_executor.invoke({"input": "Tell me about Langsmith"})
print(response1)



> Entering new AgentExecutor chain...


> Entering new None chain...


TypeError: Agent.plan() got multiple values for argument 'intermediate_steps'

In [ ]:
response2 = agent_executor.invoke({"input": "What's the paper 1605.08386 about?"})
print(response2)